# Построение регрессии

## Task 1
Давайте проанализируем данные опроса 4361 женщин из Ботсваны: **botswana.tsv**

О каждой из них мы знаем:

* сколько детей она родила (признак ceb)
* возраст (age)
* длительность получения образования (educ)
* религиозная принадлежность (religion)
* идеальное, по её мнению, количество детей в семье (idlnchld)
* была ли она когда-нибудь замужем (evermarr)
* возраст первого замужества (agefm)
* длительность получения образования мужем (heduc)
* знает ли она о методах контрацепции (knowmeth)
* использует ли она методы контрацепции (usemeth)
* живёт ли она в городе (urban)
* есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)

Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('datasets/botswana.tsv', '\t')
data.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [3]:
data.religion.value_counts()

spirit        1841
other         1080
protestant     993
catholic       447
Name: religion, dtype: int64

## Task 2
Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [4]:
data.dropna().shape

(1834, 15)

## Task 3
В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак $x_1$ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

* создать новый бинарный признак


$$\begin{equation*}
x_2 = 
 \begin{cases}
   1, &\text{$x_1$ = 'не применимо',}\\
   0, &\text{иначе;}
 \end{cases}
\end{equation*}$$

* заменить "не применимо" в $x_1$  на произвольную константу $c$, которая среди других значений $x_1$ не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида 

$$y=\beta_0 + \beta_1 x_1 + \beta_2 x_2,$$ 

на тех объектах, где $x_1$  было измерено, регрессионное уравнение примет вид

$$y=\beta_0 + \beta_1 x,$$ 

а там, где $x_1$  было "не применимо", получится

$$y=\beta_0 + \beta_1 c + \beta_2.$$ 

Выбор $c$ влияет только на значение и интерпретацию $\beta_2$ но не $\beta_1$.

Давайте используем этот метод для обработки пропусков в agefm и heduc.

1. Создайте признак nevermarr, равный единице там, где в agefm пропуски.
2. Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице XX будет мультиколлинеарность.
3. Замените NaN в признаке agefm на $c_{agefm}=0$  
4. У объектов, где nevermarr = 1, замените NaN в признаке heduc на $c_{heduc_1}=-1$ (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [5]:
data['nevermarr'] = data['agefm'].isna().astype(int)

In [6]:
data.drop('evermarr', axis=1, inplace=True)

In [7]:
data['agefm'].fillna(0, inplace=True)

In [8]:
data['heduc'] = data.apply(lambda row: -1 if row['nevermarr'] and np.isnan(row['heduc']) else row['heduc'], axis=1)

In [9]:
data['heduc'].isna().sum()

123

## Task 4
Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения ($c_{idlnchld}=-1$, $c_{heduc_2}=-2$, (значение -1 мы уже использовали), $c_{usemeth}=-1$)

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [10]:
data['idlnchld_noans'] = data['idlnchld'].isna().astype(int)
# data['idlnchld'] = data.apply(lambda row: -1 if row['idlnchld_noans'] and np.isnan(row['idlnchld']) 
#                               else row['idlnchld'], axis=1)
data['idlnchld'].fillna(-1, inplace=True)

In [11]:
data['heduc_noans'] = data['heduc'].isna().astype(int)
# data['heduc'] = data.apply(lambda row: -2 if row['heduc_noans'] and np.isnan(row['heduc']) 
#                               else row['heduc'], axis=1)
data['heduc'].fillna(-2, inplace=True)

In [12]:
data['usemeth_noans'] = data['usemeth'].isna().astype(int)
# data['usemeth'] = data.apply(lambda row: -1 if row['usemeth_noans'] and np.isnan(row['usemeth']) 
#                               else row['usemeth'], axis=1)
data['usemeth'].fillna(-2, inplace=True)

In [13]:
data.dropna(subset=['knowmeth', 'electric', 'radio', 'tv', 'bicycle'], inplace=True)

In [14]:
data.shape

(4348, 18)

In [15]:
data.shape[0] * data.shape[1]

78264

## Task 5
Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации $R^2$?  
Округлите до трёх знаков после десятичной точки.

In [16]:
import statsmodels.formula.api as smf

In [17]:
all_features = data.columns.drop('ceb')
all_features

Index(['age', 'educ', 'religion', 'idlnchld', 'knowmeth', 'usemeth', 'agefm',
       'heduc', 'urban', 'electric', 'radio', 'tv', 'bicycle', 'nevermarr',
       'idlnchld_noans', 'heduc_noans', 'usemeth_noans'],
      dtype='object')

In [18]:
model1 = smf.ols('ceb ~ ' + ' + '.join(all_features), data=data)

In [19]:
fitted_m1 = model1.fit()

In [20]:
fitted_m1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sun, 09 Sep 2018   Prob (F-statistic):               0.00
Time:                        17:31:08   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1.0263      0.212     -4.835      0.000      -1.443      -0.610
religion[T.other]         -0.0830      0.083     -1.001      0.317      -0.245       0.080
religion[T.protestant]    -0.0149      0.082     -0.181      0.857      -0.176       0.146
religion[T.spirit]        -0.0191      0.077     -0.248      0.804      -0.171       0.132
age                        0.1703      0.003     51.891      0.000       0.164       0.177
educ                      -0.0724      0.007     -9.843      0.000      -0.087      -0.058
idlnchld                   0.0760      0.011      6.923      0.000       0.054       0.098
knowmeth                   0.5564      0.121      4.580      0.000       0.318       0.795
usemeth                    0.6473      0.048     13.424      0.000       0.553       0.742
agefm                     -0.0604      0.007     -9.213      0.000      -0.073      -0.048
heduc                     -0.0551      0.008     -6.838      0.000      -0.071      -0.039
urban                     -0.2137      0.047     -4.527      0.000      -0.306      -0.121
electric                  -0.2685      0.077     -3.479      0.001      -0.420      -0.117
radio                     -0.0235      0.051     -0.461      0.645      -0.123       0.076
tv                        -0.1451      0.093     -1.566      0.118      -0.327       0.037
bicycle                    0.2139      0.050      4.260      0.000       0.115       0.312
nevermarr                 -2.2393      0.148    -15.143      0.000      -2.529      -1.949
idlnchld_noans             0.6539      0.153      4.286      0.000       0.355       0.953
heduc_noans               -0.8724      0.145     -6.026      0.000      -1.156      -0.589
usemeth_noans              1.4126      0.219      6.436      0.000       0.982       1.843
==============================================================================
Omnibus:                      224.411   Durbin-Watson:                   1.887
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              859.014
Skew:                           0.003   Prob(JB):                    2.93e-187
Kurtosis:                       5.178   Cond. No.                         361.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Task 6
Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?   
**3**

## Task 7
Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [21]:
import statsmodels.stats.api as sms

In [22]:
print(sms.het_breuschpagan(fitted_m1.resid, fitted_m1.model.exog)[1])

1.1452927633442407e-225


In [23]:
fitted_m1 = model1.fit(cov_type='HC1')

## Task 8
Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [30]:
features2 = all_features.drop(['religion', 'radio', 'tv'])
model2 = smf.ols('ceb ~ ' + ' + '.join(features2), data=data)

In [31]:
fitted_m2 = model2.fit()

In [34]:
print(sms.het_breuschpagan(fitted_m2.resid, fitted_m2.model.exog)[1])

1.1197458896531511e-228


In [35]:
fitted_m2 = model2.fit(cov_type='HC1')

In [36]:
print("F={}, p={}, k1={}".format(*fitted_m1.compare_f_test(fitted_m2)))

F=0.9192357784629898, p=0.4672305547275616, k1=5.0


/usr/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1896: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


In [37]:
fitted_m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 09 Sep 2018   Prob (F-statistic):               0.00
Time:                        17:37:31   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                0.1702      0.004     38.746      0.000       0.162       0.179
educ              -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld           0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth           0.5610      0.174      3.224      0.001       0.220       0.902
usemeth            0.6516      0.052     12.571      0.000       0.550       0.753
agefm             -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc             -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban             -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric          -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle            0.2046      0.048      4.279      0.000       0.111       0.298
nevermarr         -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans     0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans       -0.8853      0.191     -4.638      0.000      -1.259      -0.511
usemeth_noans      1.4248      0.241      5.914      0.000       0.953       1.897
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

## Task 9
Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением $c_{usemeth }=-1$, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [38]:
features3 = features2.drop(['usemeth_noans', 'usemeth'])

In [39]:
model3 = smf.ols('ceb ~ ' + ' + '.join(features3), data=data)

In [40]:
fitted_m3 = model3.fit()

In [41]:
print("F={}, p={}, k1={}".format(*fitted_m2.compare_f_test(fitted_m3)))

F=92.8905823010962, p=3.1552009480426394e-40, k1=2.0


/usr/lib/python3.7/site-packages/statsmodels/regression/linear_model.py:1896: InvalidTestWarning: F test for comparison is likely invalid with robust covariance, proceeding anyway
  InvalidTestWarning)


## Task 10
Посмотрите на доверительные интервалы для коэффициентов итоговой модели (не забудьте использовать поправку Уайта, если есть гетероскедастичность ошибки) и выберите правильные выводы.
1. Итоговая модель объясняет 63% вариации отклика
2. У женщин, знакомых с методами контрацепции, при прочих равных в среднем на 0.6 ребёнка меньше (p=0.001, 95% доверительный интервал для разницы между средними — [-0.9, -0.2])
3. У женщин, не знающих, какое количество детей идеально, в среднем на $\beta_{idlnchld\_noans} + c_{idlnchld} \beta_{idlnchld} \approx 0.58$ детей больше
4. У женщин, не знающих, какое количество детей идеально, в среднем на 0.66 ребёнка больше (p=0.002, 95% доверительный интервал — [0.2, 1.1])
5. У женщин, никогда не выходивших замуж, при прочих равных в среднем на 2.3 ребёнка меньше (p<0.001, 95% доверительный интервал для разницы между средними — [-2.6, -1.9])
6. С увеличением возраста женщины на 1 год среднее количество детей возрастает на 0.17 (p<0.001, 95% доверительный интервал — [0.16, 0.18])

**Ответ:** 3, 5, 6

In [42]:
fitted_m2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sun, 09 Sep 2018   Prob (F-statistic):               0.00
Time:                        17:53:02   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.152      0.000      -1.575      -0.565
age                0.1702      0.004     38.746      0.000       0.162       0.179
educ              -0.0729      0.007    -10.311      0.000      -0.087      -0.059
idlnchld           0.0770      0.014      5.323      0.000       0.049       0.105
knowmeth           0.5610      0.174      3.224      0.001       0.220       0.902
usemeth            0.6516      0.052     12.571      0.000       0.550       0.753
agefm             -0.0606      0.010     -6.192      0.000      -0.080      -0.041
heduc             -0.0573      0.009     -6.440      0.000      -0.075      -0.040
urban             -0.2190      0.045     -4.814      0.000      -0.308      -0.130
electric          -0.3207      0.063     -5.076      0.000      -0.445      -0.197
bicycle            0.2046      0.048      4.279      0.000       0.111       0.298
nevermarr         -2.2501      0.202    -11.158      0.000      -2.645      -1.855
idlnchld_noans     0.6565      0.216      3.043      0.002       0.234       1.079
heduc_noans       -0.8853      0.191     -4.638      0.000      -1.259      -0.511
usemeth_noans      1.4248      0.241      5.914      0.000       0.953       1.897
==============================================================================
Omnibus:                      224.096   Durbin-Watson:                   1.886
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              856.760
Skew:                           0.004   Prob(JB):                    9.06e-187
Kurtosis:                       5.175   Cond. No.                         345.
==============================================================================

Warnings:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [45]:
fitted_m2.params.idlnchld_noans - fitted_m2.params.idlnchld 

0.5794401181544879